In [1]:
import pandas as pd
import numpy as np
import re
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

import neattext as ntx
from nltk.stem.lancaster import LancasterStemmer

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, InputLayer

In [2]:
# data = pd.read_csv('datasets/data.csv')
data = pd.read_csv('datasets/less_data.csv')

In [3]:
data

,id,text_id,text,reply_count,like_count,lang,type,post_created_at,followers_count,acc_created_at,verified,statuses_count,friends_count,info_tweet,retweets,clean_text,hashtags,userhandles
0,67403135,212953800187711489,Pretty interesting: http://t.co/RtlMsOTf,1,1,en,tweet,2012-06-13 17:05:17+00:00,166.0,2009-08-20 19:40:52+00:00,False,89.0,1233.0,2,0,pretty interesting: http://tco/rtlmsotf,[],[]
1,57357366,686558302503854080,WIN a $50 Walmart Gift Card https://t.co/pXRjf...,0,0,en,tweet,2016-01-11 14:40:29+00:00,11.0,2009-07-16 15:10:05+00:00,False,787.0,40.0,0,0,win $50 walmart gift card https://tco/pxrjfawizk,[],[]
2,23867915,1305300491417591809,Cary Elwes and Robin Wright got better looking...,0,10,en,tweet,2020-09-14 00:21:03+00:00,57.0,2009-03-12 01:19:00+00:00,False,1836.0,60.0,10,0,cary elw robin wright got bet look ag husband ...,['#PrincessBrideReunion'],[]
3,26296558,674274040937185280,#Thrive4Tots Help Children in need by ord #Thr...,0,0,en,tweet,2015-12-08 17:07:13+00:00,41.0,2009-03-24 18:55:29+00:00,False,403.0,196.0,0,0,#thrive4tots help childr nee ord #thrive prtio...,"['#Thrive4Tots', '#Thrive', '#TOYS4TOTS']",['@snooki']
4,14975460,1270346530889830402,"Aunt Candace’s Shanty has a good ring it, nay?...",0,0,en,tweet,2020-06-09 13:26:29+00:00,423.0,2008-06-01 22:29:21+00:00,False,2005.0,223.0,0,0,aunt candac shanty good ring it nay publ ow bi...,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41932,32921025,1380491039216123907,French senate: Hijab Ban France https://t.co/k...,0,0,en,tweet,2021-04-09 12:01:26+00:00,704.0,2009-04-18 15:52:22+00:00,False,19212.0,641.0,0,0,french senate: hijab ban frant https://tco/kqt...,[],['@Change']
41933,71986939,24076025278,Check out this awesome site: Insidmal on I - h...,0,0,en,tweet,2010-09-10 04:24:08+00:00,6.0,2009-09-06 07:06:18+00:00,False,98.0,4.0,0,0,check awesom site: insidm http://sweevacom/go/...,['#Sweeva'],['@insidmaldesign']
41934,14792201,96418154426810368,Man I will say I really don't understand relat...,0,0,en,tweet,2011-07-28 03:14:14+00:00,2.0,2008-05-15 23:48:00+00:00,False,16.0,0.0,0,0,man understand rel,[],[]
41935,97777065,1549315866122133504,Galway RNLI rescues six people from water afte...,0,14,en,tweet,2022-07-19 08:51:02+00:00,2432.0,2009-12-19 00:04:40+00:00,False,3714.0,182.0,17,3,galway rnli rescu peopl wat group get cut tid ...,"['#RNLI', '#Galway', '#Lifeboat']",[]


In [4]:
# data['retweets'] = data['retweet_count'] + data['quote_count']
# data.drop(['retweet_count', 'quote_count'], axis=1, inplace=True)

In [5]:
def what_class(num):
    if num >= 0 and num < 10:
        return '0-10'
    elif num >= 10 and num < 60:
        return '10-60'
    elif num >= 60 and num < 100:
        return '60-100'
    elif num >= 100 and num < 200:
        return '100-200'
    elif num >= 200 and num < 500:
        return '200-500'
    elif num >= 500 and num < 1000:
        return '500-1000'
    elif num >= 1000 and num < 2000:
        return '1000-2000'
    elif num >= 2000 and num < 5000:
        return '2000-5000'
    elif num >= 5000 and num < 10000:
        return '5000-10000'
    elif num >= 10000 and num < 20000:
        return '10000-20000'
    elif num >= 20000 and num < 50000:
        return '20000-50000'
    elif num >= 50000 and num < 100000:
        return '50000-100000'
    else:
        return '100000+'

data['likes_class'] = data['like_count'].apply(lambda x: what_class(x))
data['retweets_class'] = data['retweets'].apply(lambda x: what_class(x))

In [6]:
convert_class_to_num = {'0-10':1, '10-60':2, '60-100':3, '100-200':4, '200-500':5, '500-1000':6, '1000-2000':7,\
                       '2000-5000':8, '5000-10000':9, '10000-20000':10, '20000-50000':11, '50000-100000':12, \
                        '100000+':13}

In [7]:
data['likes_class'] = data['likes_class'].apply(lambda x: convert_class_to_num[x])
data['retweets_class'] = data['retweets_class'].apply(lambda x: convert_class_to_num[x])

In [8]:
# drop_values = np.random.choice(data[(data['info_tweet'] == 0) & (data['followers_count'] < 10)]['text_id'], 4000)
drop_values = np.random.choice(data[data['info_tweet'] < 10]['text_id'], 8000)

data = data[~data['text_id'].isin(drop_values)]
data.index = range(len(data))

In [9]:
# data = data[data['type'] == 'tweet']
# data = data[data['lang'] == 'en']
# data.index = range(len(data))

In [10]:
# stemmer = LancasterStemmer()

# def remove_emojis(data):
#     emoj = re.compile("["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#         u"\U00002500-\U00002BEF"  # chinese char
#         u"\U00002702-\U000027B0"
#         u"\U00002702-\U000027B0"
#         u"\U000024C2-\U0001F251"
#         u"\U0001f926-\U0001f937"
#         u"\U00010000-\U0010ffff"
#         u"\u2640-\u2642" 
#         u"\u2600-\u2B55"
#         u"\u200d"
#         u"\u23cf"
#         u"\u23e9"
#         u"\u231a"
#         u"\ufe0f"  # dingbats
#         u"\u3030"
#                       "]+", re.UNICODE)
#     return re.sub(emoj, '', data)

# def stem_sentence(text):
#     text = [stemmer.stem(word) for word in text.split()]
    
#     return ' '.join(text)
    
    
# data['clean_text'] = data['text'].apply(lambda x: ntx.remove_stopwords(x))
# data['clean_text'] = data['clean_text'].apply(lambda x: ntx.remove_punctuations(x))
# data['clean_text'] = data['clean_text'].apply(lambda x: ntx.remove_multiple_spaces(x))
# data['clean_text'] = data['clean_text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
# data['clean_text'] = data['clean_text'].apply(lambda x: remove_emojis(x))

# data['hashtags'] = data['clean_text'].apply(lambda x: ntx.extract_hashtags(x))
# data['urls'] = data['clean_text'].apply(lambda x: ntx.extract_urls(x))
# data['userhandles'] = data['clean_text'].apply(lambda x: ntx.extract_userhandles(x))
# # data['clean_text'] = data['clean_text'].apply(lambda x: ntx.remove_hashtags(x))
# data['clean_text'] = data['clean_text'].apply(lambda x: ntx.remove_urls(x))
# data['clean_text'] = data['clean_text'].apply(lambda x: ntx.remove_userhandles(x))

# data['clean_text'] = data['clean_text'].apply(lambda x: stem_sentence(x))

In [11]:
# print(data['clean_text'][0])
# print(data['text'][0])

In [12]:
# def number_in_text(text, number):
#     for i in number:
#         if i in text:
#             return True
#     return False

# a = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
# print('Clean text:', data['clean_text'].apply(lambda x: number_in_text(x,a)).sum(), 'sentences with numbers')
# print('Text:', data['text'].apply(lambda x: number_in_text(x,a)).sum(), 'sentences with numbers')

In [13]:
# less_data = pd.DataFrame(columns=data.columns)

# ind = np.random.randint(0, data.index[-1]+1, 42000)
# less_data = data.loc[ind]

# less_data.to_csv('datasets/less_data.csv', index=False)

In [14]:
y = data['likes_class'].copy()
y = y.to_numpy().astype(np.float16)

In [15]:
########## TEXT


vectorizer = TfidfVectorizer()
# vectorizer = CountVectorizer(binary=True)

vectorized_text = vectorizer.fit_transform(data['clean_text'])
vectorized_text = vectorized_text.astype(np.float32)
# vectorized_text = vectorized_text.astype(np.int32)
vectorized_text = vectorized_text.toarray()


vectorized_text_with_followers = np.insert(vectorized_text, 0, data['followers_count'], axis=1) 
del vectorized_text

In [16]:
# to_input = vectorized_text_with_followers

In [17]:
scaler = StandardScaler()
to_input = scaler.fit_transform(vectorized_text_with_followers)
to_input = to_input.astype(np.float16)
del vectorized_text_with_followers

In [18]:
# to_input_train, to_input_test, y_train, y_test = train_test_split(to_input, y, test_size=.2)
# del to_input
# del y

# CLASSIFICATION MODELS

In [19]:
scores = []
for i in range(5):
    to_input_train, to_input_test, y_train, y_test = train_test_split(to_input, y, test_size=.2)
    del to_input
    del y

    model = LogisticRegression()
    model.fit(to_input_train, y_train)
    scores.append(model.score(to_input_test, y_test))
    
    to_input = np.concatenate((to_input_train, to_input_test))
    y = np.concatenate((y_train, y_test))
    
    del to_input_train
    del to_input_test
    del y_train
    del y_test

/home/danilo/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/danilo/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

In [29]:
np.sum(scores) / len(scores)

0.8889169675090253

In [33]:
# print(np.where(y == 3))
n = 18

print(model.predict(to_input[n][np.newaxis, :]))
print(y[n])

[3.]
3.0


# NEURAL NETWORK BAD IDEA, BECAUSE WE NEED SUCH A POWER COMPUTER

In [22]:
# model = Sequential()
# model.add(LSTM(50, return_sequences=True, input_shape=(1,to_input_train.shape[-1])))
# model.add(LSTM(50))
# model.add(Dense(32))
# model.add(Dense(1, activation='softmax'))

In [23]:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
# model.fit(to_input_train[:, np.newaxis, :], y_train, batch_size=32, epochs=10)

In [25]:
# model.save('model.h5')

## HASHTAGS

In [26]:
# def take_unique_hashtags(ser):
#     unique = []
#     for lis in ser:
#         for hashtag in lis:
#             if hashtag not in unique:
#                 unique.append(hashtag)
    
#     return unique

# data['hashtags'] = clear_hashtags(data['hashtags'].copy())
# unique_hashtags = take_unique_hashtags(data['hashtags'])

In [27]:
# ########## HASHTAGS


# vectorize_hash = TfidfVectorizer()

# # vectorized_hash_fited = vectorize_hash.fit_transform(data['hashtags'])
# # vectorized_hash_fited = vectorized_hash_fited.astype(np.float16)
# # vectorized_hash_fited = vectorized_hash_fited.toarray()

# vectorize_hash.fit(unique_hashtags)

In [28]:
# len_of_hash_uniq = len(vectorize_hash.get_feature_names_out())

# def convert_hashtag_to_vector(hashtags, vectorizer, len_of_features):
#     ones = np.ones((len_of_features))
#     if len(hashtags) == 1:
#         ind = np.argmax(vectorizer.transform(hashtags))
#         ones[ind] = 1
#     elif len(hashtags) >= 2:
#         indexes = []
#         for hashtag in hashtags:
#             indexes.append(np.argmax(vectorizer.transform([hashtag])))
#         for index in indexes:
#             ones[index] = 1
        
#     return ones

        
# # vectorize_hash.transform(['#hello']).toarray()
# data['hashtags'] = data['hashtags'].apply(lambda x: convert_hashtag_to_vector(x, vectorize_hash, len_of_hash_uniq))